In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [2]:
from bnp.branch import BranchAndPrice
from bnp.nodes import Node

In [3]:
dataset = pd.read_csv("data.txt", sep=" ")
print(dataset.head())

      w   d
0  75.0  38
1  71.3  44
2  67.0  30
3  61.4  41
4  58.0  36


In [4]:
def solve_knapsack(W, w, duals):
    return linprog(-duals, A_ub=np.atleast_2d(w), b_ub=np.atleast_1d(W),
                   bounds=(0, np.inf), integrality=1)

In [5]:
W = 100.0

w = dataset.w.values
b = dataset.d.values
A = np.eye(dataset.shape[0]) * (W // w)
# c = W % w
c = np.ones_like(w)

def pricing_rule(node: Node):
    duals = -node.sol.ineqlin.marginals
    price_sol = solve_knapsack(W, w, duals)
    if price_sol.fun < -1 -1e-6:
        node.A_ub = np.hstack((node.A_ub, -price_sol.x.reshape((-1, 1))))
        node.c = np.append(node.c, 1)
        node.lb = np.append(node.lb, 0)
        node.ub = np.append(node.ub, np.inf)
        node.integrality = np.append(node.integrality, True)
        print(f"pricing: {price_sol.fun}")
        return True
    else:
        print(f"No new columns")
        return False
        

# sol_dual = linprog(-b, A_ub=A.T, b_ub=c, bounds=(0, None))

In [6]:
milp = BranchAndPrice(branching_rule="frac")

In [7]:
sol = milp(c, pricing_rule=pricing_rule, A_ub=-A, b_ub=-b, lb=0, ub=np.inf, max_iter=100, verbose=True, int_tol=1e-2)

pricing: -1.5909090909090908
pricing: -1.5353535353535357
pricing: -1.5333333333333332
pricing: -1.5
pricing: -1.5151515151515151
pricing: -1.5
pricing: -1.511111111111111
pricing: -1.5
pricing: -1.4
pricing: -1.4126984126984126
pricing: -1.4333333333333333
pricing: -1.3333333333333333
No new columns
First solution is feasible but not integer
Feasible below incumbent 1: 334.1666666666667
Feasible below incumbent 2: 334.1666666666667
Feasible below incumbent 3: 334.1666666666667
Feasible below incumbent 4: 334.1666666666667
Feasible below incumbent 5: 334.33333333333326
Feasible below incumbent 6: 334.3333333333333
pricing: -1.4444444444444444
No new columns
Feasible below incumbent 7: 335.0
Feasible below incumbent 8: 335.1666666666667
Feasible below incumbent 9: 335.22222222222223
pricing: -1.3333333333333333
No new columns
Feasible below incumbent 10: 336.0
No new columns
New best sol 11: 336.0
Fathom node 11 due to poor parent node relaxation
Feasible below incumbent 12: 335.5
prici

In [13]:
sol

        message: MIP gap tolance reached 0.0000
        success: True
         status: 0
            fun: 334.0
              x: [ 3.800e+01  4.400e+01 ...  2.000e+00  6.000e+00]
            nit: 14
          lower:  residual: [ 3.800e+01  4.400e+01 ...  2.000e+00
                              6.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  2.700e+01
                              0.000e+00]
                 marginals: [-1.000e+00 -1.000e+00 ... -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
       explored: 34
       fathomed:

In [11]:
id(sol.node.parent.A_ub)

2162122738832

In [12]:
id(sol.node.A_ub)

2162122738832

In [8]:
sol.node.A_ub.shape

(19, 34)

In [9]:
sol.node.parent.sol.x

array([38.        , 44.        , 30.        , 41.        , 10.14102564,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  3.97435897,
       21.        ,  1.70512821,  5.32051282,  8.97435897, 29.32051282,
       14.52564103, 33.        , 14.67948718, 12.5       ,  1.35897436,
       24.5       ])

In [10]:
sol.node.sol.x

array([38.        , 44.        , 30.        , 41.        , 10.14102564,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  3.97435897,
       21.        ,  1.70512821,  5.32051282,  8.97435897, 29.32051282,
       14.52564103, 33.        , 14.67948718, 12.5       ,  1.35897436,
       24.5       ])

In [11]:
sol.node.integrality

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1])

In [12]:
res = sol.node._get_residuals()
sol.node._minimum_violation(res)

0

In [13]:
mask = (res > sol.node.int_tol) & sol.node.integrality.astype(bool)

In [14]:
mask

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True])

In [15]:
sol.node.children[0].lb

array([38.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [16]:
sol.node.lb

array([38.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [100]:
W = 100.0

w = dataset.w.values
b = dataset.d.values
A = np.eye(dataset.shape[0]) * (W // w)
c = W % w
# c = np.ones_like(w)

sol_dual = linprog(-b, A_ub=A.T, b_ub=c, bounds=(0, None))

for _ in range(20):
    price_sol = solve_knapsack(W, w, sol_dual.x)
    new_col = price_sol.x
    new_c = W - new_col.dot(w)
    # new_c = 1
    if price_sol.fun < -new_c:
        print(f"Iteration {_}: price{price_sol.fun}; c: {new_c}")
        print(f"New column {new_col}")
        A = np.hstack((A, new_col.reshape((-1, 1))))
        c = np.append(c, new_c)
        sol_dual = linprog(-b, A_ub=A.T, b_ub=c, bounds=(0, None))
    else:
        sol = linprog(c, A_ub=-A, b_ub=-b, bounds=(0, np.inf), integrality=1)
        print(f"Iteration {_}: price{price_sol.fun}; c: {new_c}")
        break

sol = linprog(c, A_ub=-A, b_ub=-b, bounds=(0, np.inf), integrality=1)
print(sol.x.sum())

Iteration 0: price-60.090909090909086; c: 1.0
New column [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2. 0. 0. 0. 1. 0. 0.]
Iteration 1: price-55.5; c: 5.5
New column [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0.]
Iteration 2: price-51.515151515151516; c: 0.0
New column [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 2. 0. 0.]
Iteration 3: price-48.93333333333334; c: 0.6000000000000085
New column [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
Iteration 4: price-43.199999999999996; c: 3.200000000000003
New column [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0. -0.  0.  0.  0.  0.
  1.]
Iteration 5: price-39.733333333333334; c: 6.400000000000006
New column [0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Iteration 6: price-33.93333333333334; c: 0.5999999999999943
New column [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
Iteration 7: price-28.922222222222224; c: 6.700000000000003
New column [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 

In [96]:
sol_dual.x

array([25. ,  6.7,  0.6,  6.4,  3.2,  0. ,  0. ,  5.5,  1. ,  0. ,  0.6,
       -0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ])

In [95]:
new_col

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [94]:
new_col.dot(w)

75.0

In [88]:
A = np.vstack((A, np.zeros(A.shape[1])))
b = np.append(b, -9)
A[-1, 12] = -1
sol = linprog(c, A_ub=-A, b_ub=-b, bounds=(0, np.inf), integrality=0)
print(sol.x)

[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.   41.   36.   36.   36.
 41.   30.   44.   35.    0.   38.    0.    9.25  0.    0.    0.    0.
  0.    0.    0.  ]


In [84]:
sol.ineqlin.marginals[:-1]

array([-4.9, -0. , -6.4, -3. , -0.2, -0. , -5.5, -1. , -2.3, -0. , -0. ,
       -0. , -4.9, -0. , -0. , -0. , -0. , -0. , -0. ])

In [85]:
price_sol = solve_knapsack(W, w, -sol.ineqlin.marginals[:-1])
new_c = W % new_col.dot(w)

In [86]:
price_sol.fun

-19.599999999999994

In [87]:
new_c

19.599999999999994

In [74]:
W = 100.0

w = dataset.w.values
b = dataset.d.values
A = np.eye(dataset.shape[0]) * (W // w)
# c = W % w
c = np.ones_like(w)

sol_dual = linprog(-b, A_ub=A.T, b_ub=c, bounds=(0, None))

for _ in range(20):
    price_sol = solve_knapsack(W, w, sol_dual.x)
    # new_c = W % new_col.dot(w)
    new_c = 1
    if price_sol.fun < -new_c:
        print(f"Iteration {_}: price{price_sol.fun}; c: {new_c}")
        new_col = price_sol.x
        A = np.hstack((A, new_col.reshape((-1, 1))))
        c = np.append(c, new_c)
        sol_dual = linprog(-b, A_ub=A.T, b_ub=c, bounds=(0, None))
    else:
        sol = linprog(c, A_ub=-A, b_ub=-b, bounds=(0, np.inf), integrality=1)
        break

print(sol)

Iteration 0: price-1.5909090909090908; c: 1
Iteration 1: price-1.5353535353535357; c: 1
Iteration 2: price-1.5333333333333332; c: 1
Iteration 3: price-1.5; c: 1
Iteration 4: price-1.5151515151515151; c: 1
Iteration 5: price-1.5; c: 1
Iteration 6: price-1.511111111111111; c: 1
Iteration 7: price-1.4999999999999998; c: 1
Iteration 8: price-1.4; c: 1
Iteration 9: price-1.4126984126984126; c: 1
Iteration 10: price-1.4333333333333331; c: 1
Iteration 11: price-1.3333333333333333; c: 1
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 334.0
              x: [ 3.800e+01  4.400e+01 ...  1.000e+00  3.500e+01]
            nit: -1
          lower:  residual: [ 3.800e+01  4.400e+01 ...  1.000e+00
                              3.500e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
    

In [75]:
sol.x.sum()

334.0

In [65]:
new_c

25.0

In [53]:
price_sol.x

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [33]:
price_sol = solve_knapsack(W, w, sol_dual.x)
new_col = price_sol.x
new_c = W % new_col.dot(w)

In [45]:
red_cost

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [43]:
red_cost.dot(price_sol.x).dot(price_sol.x)

0.0

In [44]:
red_cost.dot(sol.x)

0.0

In [41]:
price_sol.x.dot(c)

94.39999999999999

In [42]:
sol_dual.x.dot(new_col)

60.090909090909086

In [36]:
new_col.shape

(19,)

In [35]:
new_col.dot(c)

94.39999999999999

In [31]:
red_cost

0.0

In [30]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 13948.0
              x: [ 3.800e+01  4.400e+01 ...  3.000e+00  4.000e+00]
            nit: -1
          lower:  residual: [ 3.800e+01  4.400e+01 ...  3.000e+00
                              4.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  1.000e+00
                              1.100e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 13948.0
        mip_gap: 0

In [28]:
for i in (c - A.T.dot(sol_dual.x)):
    print(i)

0.0
22.0
32.400000000000006
32.199999999999996
38.8
46.2
47.0
43.5
48.8
3.3999999999999915
5.799999999999972
10.599999999999998
19.599999999999994
2.8000000000000043
13.0
1.0
5.400000000000004
1.6000000000000085
1.0000000000000053
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

In [22]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 12023.4
              x: [ 3.800e+01  4.400e+01 ...  4.000e+00  3.600e+01]
            nit: -1
          lower:  residual: [ 3.800e+01  4.400e+01 ...  4.000e+00
                              3.600e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  1.000e+00
                              1.100e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
 mip_node_count: 1
 mip_dual_bound: 12023.4
        mip_gap: 0

In [24]:
_

0

In [23]:
np.sum(c - A.T.dot(sol_dual.x))

68.89090909090908

In [18]:
A.T.dot(sol_dual.x) - c

array([  0. , -22. , -32.4, -32.2, -38.8, -46.2, -47. , -43.5, -48.8,
        -3.4,  -5.8, -10.6, -19.6,  -2.8, -13. ,  -1. ,  -5.4,  -1.6,
        -1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ])

In [16]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 1927.3000000000006
              x: [ 3.800e+01  0.000e+00 ...  3.000e+01  4.400e+01]
            nit: -1
          lower:  residual: [ 3.800e+01  0.000e+00 ...  3.000e+01
                              4.400e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  9.000e+00
                              3.100e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
 mip_node_count: 1
 mip_dual_bound: 1927.3000000000

In [13]:
price_sol.fun

-25.0

In [9]:
new_c

0.6000000000000085

In [6]:
sol = linprog(c, A_ub=-A, b_ub=-b, bounds=(0, np.inf), integrality=1)

In [7]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 8270.300000000001
              x: [ 3.800e+01  4.400e+01 ...  4.100e+01  4.900e+01]
            nit: -1
          lower:  residual: [ 3.800e+01  4.400e+01 ...  4.100e+01
                              4.900e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  1.000e+00
                              1.100e+01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
 mip_node_count: 1
 mip_dual_bound: 8270.30000000000

In [37]:
sol.x

array([38.,  0.,  0., ...,  0.,  0.,  0.])

In [34]:
price_sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -25.0
              x: [ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: -1
          lower:  residual: [ 1.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 2.500e+01]
                 marginals: [ 0.000e+00]
 mip_node_count: 1
 mip_dual_bound: -25.0
        mip_gap: 0.0

In [25]:
sol = linprog(c, A_ub=-A, b_ub=-b, bounds=(0, np.inf))

In [26]:
sol_dual = linprog(-b, A_ub=A.T, b_ub=c, bounds=(0, None))

In [27]:
solve_knapsack(W, w, sol_dual.x)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -60.090909090909086
              x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: -1
          lower:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 1.000e+00]
                 marginals: [ 0.000e+00]
 mip_node_count: 1
 mip_dual_bound: -60.090909090909086
        mip_gap: 0.0

In [12]:
sol_dual.fun

-13916.306060606057

In [11]:
sol_dual.x

array([25.        , 28.7       , 33.        , 38.6       , 42.        ,
       46.2       , 47.        , 49.        , 49.8       ,  1.13333333,
        2.53333333,  3.53333333,  4.9       ,  0.46666667,  2.16666667,
        0.11111111,  0.49090909,  0.13333333,  0.06666667])

In [9]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 13916.306060606057
              x: [ 3.800e+01  4.400e+01 ...  2.917e+00  3.267e+00]
            nit: 0
          lower:  residual: [ 3.800e+01  4.400e+01 ...  2.917e+00
                              3.267e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [-2.500e+01 -2.870e+01 ... -1.333e-01
                             -6.667e-02]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_